## Local DuckGPT Cookbook

This notebook shows common recipes for working with the local data using DuckGPT

### Create the conversation

Create the conversation and load the database if needed 

In [ ]:
from duckgpt.local.conversation import duckdb_agent, get_duckgpt_conversation
from duckgpt.local.load_tables import load_tables

duckgpt_conversation = get_duckgpt_conversation()

LOAD_DATABASE = True
if LOAD_DATABASE:
    load_tables(duckdb_agent=duckdb_agent)

### Chat with DuckGPT

You can chat with a conversation using:
1. The `print_response()` function that prints the reponse
2. The `chat()` function that returns the response

In [2]:
duckgpt_conversation.print_response("Which actor has the highest average movie rating?")

Output()

INFO     Running: SHOW TABLES

INFO     Running: DESCRIBE Movies

INFO     Running: WITH SplitActors AS (                                                                            
             SELECT TRIM(UNNEST(string_to_array(Actors, ','))) AS Actor,                                           
                    Rating                                                                                         
             FROM Movies                                                                                           
             WHERE Actors IS NOT NULL AND Rating IS NOT NULL                                                       
         ),                                                                                                        
                                                                                                                   
         AvgRatings AS (                                                                                           
             SELECT Actor,                                                                                         
                    AVG(Rating) AS AvgRating                                                                       
             FROM SplitActors                                                                                      
             GROUP BY Actor                                                                                        
         )                                                                                                         
                                                                                                                   
         SELECT Actor,                                                                                             
                AvgRating                                                                                          
         FROM AvgRatings                                                                                           
         ORDER BY AvgRating DESC                                                                                   
         LIMIT 1

In [ ]:
rating_histogram_response = duckgpt_conversation.chat("Give me a histogram of movies by rating, decide the best bucket size?", stream=False)
print(rating_histogram_response)

### Run a DuckDb Query

You can run a DuckDb in 2 ways:
1. Using the DuckDb connection directly
2. Using the `run_query()` function of the duckdb_agent

In [ ]:
duckdb_agent.run_query("SELECT * FROM Movies LIMIT 5")

In [8]:
duckdb_agent.connection.sql("SELECT * FROM Movies LIMIT 5").show()

┌───────┬──────────────────────┬──────────────────────┬───┬────────┬────────────────────┬───────────┐
│ Rank  │        Title         │        Genre         │ … │ Votes  │ Revenue (Millions) │ Metascore │
│ int64 │       varchar        │       varchar        │   │ int64  │       double       │   int64   │
├───────┼──────────────────────┼──────────────────────┼───┼────────┼────────────────────┼───────────┤
│     1 │ Guardians of the G…  │ Action,Adventure,S…  │ … │ 757074 │             333.13 │        76 │
│     2 │ Prometheus           │ Adventure,Mystery,…  │ … │ 485820 │             126.46 │        65 │
│     3 │ Split                │ Horror,Thriller      │ … │ 157606 │             138.12 │        62 │
│     4 │ Sing                 │ Animation,Comedy,F…  │ … │  60545 │             270.32 │        59 │
│     5 │ Suicide Squad        │ Action,Adventure,F…  │ … │ 393727 │             325.02 │        40 │
├───────┴──────────────────────┴──────────────────────┴───┴────────┴──────────────